In [1]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

In [2]:
# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import os
import requests
import tarfile
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np

import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.distribute_eia923 as distribute_eia923

C:\Users\Greg\miniconda3\envs\hourly_egrid\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Specify the year for analysis

In [4]:
year = 2020

# 1. Download data

 - Downloads the pre-cleaned PUDL versions of EIA-923, EIA-860, and EPA CEMS data  
 - Downloads EPA eGRID data  
 - Downloads EIA-930 data  
 - Downloads the EPA Power Sector Data Crosswalk

TODO
- [x] The code for downloading the files could probably be made into functions
- [ ] Investigate other packages besides `requests` that would download these files faster

In [4]:
############### PUDL Database ######################

load_data.download_pudl_data(zenodo_url = 'https://zenodo.org/record/6349861/files/pudl-v0.6.0-2022-03-12.tgz')

################# eGRID data #########################

# the 2019 and 2020 data appear to be hosted on different urls
egrid_files_to_download = ['https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx', 
                           'https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx']

load_data.download_egrid_files(egrid_files_to_download)

############# EIA-930 data #####################

load_data.download_eia930_data(years_to_download=[year])

########## Power Sector Data Crosswalk #############
# NOTE: Check for new releases at https://github.com/USEPA/camd-eia-crosswalk

load_data.download_epa_psdc(psdc_url='https://github.com/USEPA/camd-eia-crosswalk/releases/download/v0.2.1/epa_eia_crosswalk.csv')


PUDL data already downloaded
egrid2019_data.xlsx already downloaded
egrid2020_data.xlsx already downloaded
2020_Jan_Jun data already downloaded
2020_Jul_Dec data already downloaded
epa_eia_crosswalk.csv already downloaded


# 1. Clean EIA-923 Generation and Fuel Data at the Monthly Level

In [136]:
# Distribute net generation and heat input data reported by the three different EIA-923 tables
# NOTE: this code was copied and modified from `pudl.analysis.allocate_net_gen`
# NOTE: this code allocates net generation based on the proportion of net generation reported, rather than by nameplate capacity (which eGRID does)
# NOTE: the code was modified to perform the allocation on a monthly basis, rather than an annual basis

# HIGH PRIORITIES
# DONE: remove non-grid connected plants from this dataframe
# DONE: Denormalize data by balancing authority/state. BA assignment from EIA-860
# TODO: look into whether net_gen from generation table should be preserved if available
# TODO: figure out what happens when each table has different values for net gen (allocate difference, or take precedence)
# DONE: add nuclear generators to this list
# TODO: Remove any plants located in Puerto Rico

# FUEL ASSIGNMENT
# DONE: calculate total emissions from gf based on fuel and heat input and distribute in addition to net generation and fuel consumed
# TODO: when aggregating back to generator records, keep the fuel type that accounted for most heat input
# TODO: denormalize data by fuel type both primary fuel type by generator, and primary fuel by plant (assuming that's how reported to ISOs)
# primary fuel type is currently assigned based on the annual primary fuel type. This should be changed to assign base on monthly fuel type

# NEXT PRIORITIES
# TODO: allocate heat input data from boiler_fuel_eia923() See: https://github.com/catalyst-cooperative/pudl/pull/1096
# TODO: Also distribute heat input for electricity consumption

# LOWER PRIORITIES
# TODO: fix allocation of net generation when reported net generation is negative?
# TODO: investigate generators for which frac column is not adding to 1.0

gen_fuel_allocated = distribute_eia923.allocate_gen_fuel_by_gen(year=year)

Removing 0 plants that are not grid-connected
Removing 0 plants that are not grid-connected
Removing 1 plants that are not grid-connected


c:\Users\Greg\GitHub\hourly-egrid\notebooks\../../hourly-egrid\src\distribute_eia923.py:924: UserWarning: Ooopsies. You got 1495 records where the 'frac' column isn't adding up to 1 for each 'IDX_PM_FUEL' group. Check 'calc_allocation_fraction()'
        plant_id_eia prime_mover_code energy_source_code report_date  frac  \
1326             141               ST                DFO  2020-01-01   3.0   
1348             141               ST                DFO  2020-12-01   3.0   
1350             141               ST                 NG  2020-01-01   3.0   
1372             141               ST                 NG  2020-12-01   3.0   
3259             350               ST                 NG  2020-03-01   2.0   
...              ...              ...                ...         ...   ...   
133878         60903               CT                 NG  2020-05-01   2.0   
133885         60903               CA                 NG  2020-09-01   2.0   
133886         60903               CT              

In [133]:
# for which plants are we still missing co2 data?
gen_fuel_allocated[(gen_fuel_allocated['co2_mass_tons'].isna()) & (gen_fuel_allocated['fuel_consumed_mmbtu'] > 0)]

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_tons,ba_code,state,energy_source_code


# 2. Clean Hourly Data from CEMS
There are three broad categories of plants based on their CAMD reporting status:
1. Units that report to CAMD year-round (for these plants, emissions data is used directly from CEMS)
2. Units that only report to CAMD during the ozone season (May-Sept) (for these units, non-ozone season data is taken from EIA 923)
3. Units that do not report to CAMD (generally fossil units < 25MW and non-fossil generators)

There are also certain plants that report to CAMD but do not procuce electricty for the grid, and need to be removed from the CEMS data:
- Non grid connected plants
- Steam-only plants


In [ ]:
# NOTE: all of the functions in this section could be run by calling clean_cems()
#cems = data_cleaning.clean_cems(year)

In [135]:
# load the CEMS data
cems = load_data.load_cems_data(year)

# remove non-grid connected plants
cems = data_cleaning.remove_non_grid_connected_plants(cems)

# manually remove steam-only units
cems = data_cleaning.manually_remove_steam_units(cems)

# remove plants that only report steam generation and no electrical generation
# NOTE: keeping steam only plants for now
#cems = data_cleaning.remove_heating_only_plants(cems)

# add a report date
cems = data_cleaning.add_report_date(cems)

# identify cems reporting status (full year or partial year)
# NOTE: this information is not really useful yet, so we are not going to run this to save time
#cems = data_cleaning.determine_cems_reporting_status(cems)

# TODO: identify and remove any hourly values that appear to be outliers

# add a fuel type to each observation
cems = data_cleaning.assign_fuel_type_to_cems(cems,year)

# fill in missing hourly emissions data using the fuel type and heat input
cems = data_cleaning.fill_cems_missing_co2(cems, year)

# remove any observations from cems where zero operation is reported for an entire month
# although this data could be considered to be accurately reported, let's remove it so that we can double check against the eia data
# TODO: check if any of these observations are from geothermal generators
cems = data_cleaning.remove_cems_with_zero_monthly_data(cems)

# identify any remaining missing values
# TODO: Try to identify fuel types
still_missing_co2_data = list(cems[cems['co2_mass_tons'].isnull()]['cems_id'].unique())
print(f"Unable to calculate emissions for the following plants_units: {still_missing_co2_data}")
cems_missing_fuel = cems[cems['co2_mass_tons'].isnull()]

# For now, lets drop these from the data
cems = cems[~cems['cems_id'].isin(still_missing_co2_data)]

cems_missing_fuel

Removing 45 plants that are not grid-connected
removing 7329216 observations from cems for unit-months where no data reported
Unable to calculate emissions for the following plants_units: []


,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,co2_mass_measurement_code,plant_id_epa,unit_id_epa,report_date,energy_source_code


In [137]:
# identify which units are still missing a fuel type assignment
cems[cems['energy_source_code'].isnull()]

,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,co2_mass_measurement_code,plant_id_epa,unit_id_epa,report_date,energy_source_code
445104,54096,X026,54096_X026,2020-07-01 06:00:00+00:00,1.0,23.0,23.0,NaN,310.899994,20.164635,<NA>,54096,88302,2020-07-01,NaN
445105,54096,X026,54096_X026,2020-07-01 07:00:00+00:00,1.0,23.0,23.0,NaN,311.000000,20.171121,<NA>,54096,88302,2020-07-01,NaN
445106,54096,X026,54096_X026,2020-07-01 08:00:00+00:00,1.0,23.0,23.0,NaN,310.600006,20.145178,<NA>,54096,88302,2020-07-01,NaN
445107,54096,X026,54096_X026,2020-07-01 09:00:00+00:00,1.0,23.0,23.0,NaN,314.600006,20.404614,<NA>,54096,88302,2020-07-01,NaN
445108,54096,X026,54096_X026,2020-07-01 10:00:00+00:00,1.0,23.0,23.0,NaN,316.000000,20.495416,<NA>,54096,88302,2020-07-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34770643,4125,8,4125_8,2021-01-01 01:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.000000,<NA>,4125,2621,2020-12-01,NaN
34770644,4125,8,4125_8,2021-01-01 02:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.000000,<NA>,4125,2621,2020-12-01,NaN
34770645,4125,8,4125_8,2021-01-01 03:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.000000,<NA>,4125,2621,2020-12-01,NaN
34770646,4125,8,4125_8,2021-01-01 04:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.000000,<NA>,4125,2621,2020-12-01,NaN


In [138]:
# flag any generator-months for which we already have cems data
# NOTE: there is still an issue identifying for which generators we have cems data because of incomplete mapping
#gen_fuel_allocated = gen_fuel_allocated.drop(columns=['data_source'])
gen_fuel_allocated = data_cleaning.identify_emissions_data_source(cems, gen_fuel_allocated)

# create a separate dataframe containing only the generators for which we do not have CEMS data
monthly_eia_data_to_distribute = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'eia_only') & ~(gen_fuel_allocated['fuel_consumed_mmbtu'].isna())]

# what percent of generators are in CEMS vs not
gen_fuel_allocated.groupby('data_source').sum() / gen_fuel_allocated.groupby('data_source').sum().sum(axis=0)

,plant_id_eia,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_tons
data_source,,,,,
cems,0.122714,0.572436,0.525128,0.547364,0.83372
eia_only,0.877286,0.427564,0.474872,0.452636,0.16628


In [140]:
# convert hourly gross generation to net generation
#cems = cems.drop(columns=['net_generation_mwh','gross_to_net_ratio','net_gen_method'])
cems = data_cleaning.convert_gross_to_net_generation(cems, gen_fuel_allocated)

# for generators where there is heat input but no gross generation reported, impute hourly net generation based on reported EIA values
cems = data_cleaning.impute_missing_hourly_net_generation(cems, gen_fuel_allocated)

# what percent of net generation was allocated using each method?
cems.groupby('net_gen_method', dropna=False).sum()['net_generation_mwh'] / cems['net_generation_mwh'].sum()

net_gen_method
annual_regression            0.002535
imputed_from_heat_content    0.000852
monthly_ratio                0.994018
net_equals_gross             0.002595
Name: net_generation_mwh, dtype: float32

In [141]:
# add information about the balancing authority 
#cems = cems.drop(columns=['ba_code','state'])
cems = data_cleaning.assign_ba_code_to_plant(cems, year)

# 3. Compare Monthly data from CEMS/EIA-923
We have now identified all plants that report the full year to CEMS, and all plants that report a partial year. We will now use the EIA-923 data to fill in the missing pieces.


In [11]:
# TODO: Compare the data reported by both sources
# for plants where there is data reported in cems, see how off it is from data reported in eia
cems_plant_monthly = cems.groupby(['plant_id_eia','report_date']).sum()[['net_generation_mwh','heat_content_mmbtu','co2_mass_tons']].reset_index().rename(columns={'heat_content_mmbtu':'fuel_consumed_mmbtu'})
gf_plant_monthly = gen_fuel_allocated.groupby(['plant_id_eia','report_date']).sum().reset_index()
gf_plant_monthly = gf_plant_monthly.merge(cems_plant_monthly, how='inner', on=['plant_id_eia','report_date'], suffixes=("_eia",'_cems'))

gf_plant_monthly['pctdiff_gen'] = ((gf_plant_monthly['net_generation_mwh_cems'].replace(0,0.1) - gf_plant_monthly['net_generation_mwh_eia'].replace(0,0.1)) / gf_plant_monthly['net_generation_mwh_eia'].replace(0,0.1)).round(3)
gf_plant_monthly['pctdiff_fuel'] = ((gf_plant_monthly['fuel_consumed_mmbtu_cems'].replace(0,0.1) - gf_plant_monthly['fuel_consumed_mmbtu_eia'].replace(0,0.1)) / gf_plant_monthly['fuel_consumed_mmbtu_eia'].replace(0,0.1)).round(3)
gf_plant_monthly['pctdiff_co2'] = ((gf_plant_monthly['co2_mass_tons_cems'].replace(0,0.1) - gf_plant_monthly['co2_mass_tons_eia'].replace(0,0.1)) / gf_plant_monthly['co2_mass_tons_eia'].replace(0,0.1)).round(3)

gf_plant_monthly.sort_values(by='pctdiff_gen')

,plant_id_eia,report_date,net_generation_mwh_eia,fuel_consumed_mmbtu_eia,fuel_consumed_for_electricity_mmbtu,co2_mass_tons_eia,net_generation_mwh_cems,fuel_consumed_mmbtu_cems,co2_mass_tons_cems,pctdiff_gen,pctdiff_fuel,pctdiff_co2
11124,55641,2020-09-01,-505.00,0.0,0.0,0.00000,261056.609375,1.817727e+06,108023.497268,-517.944,1.817727e+07,1080233.973
11125,55641,2020-10-01,-972.00,0.0,0.0,0.00000,370347.375000,2.564216e+06,152387.475818,-382.016,2.564216e+07,1523873.758
11126,55641,2020-11-01,-1002.00,0.0,0.0,0.00000,141516.921875,9.632251e+05,57242.498871,-142.234,9.632250e+06,572423.989
1806,1599,2020-11-01,-751.00,7648.0,7648.0,446.94912,5321.660156,5.219627e+04,3527.146240,-8.086,5.825000e+00,6.892
4379,6035,2020-11-01,-128.92,10804.0,10804.0,631.38576,731.080017,1.051120e+04,624.776217,-6.671,-2.700000e-02,-0.010
...,...,...,...,...,...,...,...,...,...,...,...,...
3951,3804,2020-06-01,0.00,0.0,0.0,0.00000,329370.281250,2.359999e+06,140251.348182,3293701.812,2.359999e+07,1402512.482
4663,6112,2020-01-01,0.00,0.0,0.0,0.00000,337367.343750,2.632619e+06,156451.644281,3373672.438,2.632619e+07,1564515.443
10320,55382,2020-01-01,0.00,0.0,0.0,0.00000,521570.031250,3.892977e+06,231354.493562,5215699.312,3.892977e+07,2313543.936
4753,6155,2020-08-01,0.00,0.0,0.0,0.00000,773619.312500,7.606572e+06,797778.599762,7736192.125,7.606571e+07,7977784.998


In [12]:
# investigate single plants
gf_plant_monthly[gf_plant_monthly['plant_id_eia'] == 568]

,plant_id_eia,report_date,net_generation_mwh_eia,fuel_consumed_mmbtu_eia,fuel_consumed_for_electricity_mmbtu,co2_mass_tons_eia,net_generation_mwh_cems,fuel_consumed_mmbtu_cems,co2_mass_tons_cems,pctdiff_gen,pctdiff_fuel,pctdiff_co2
630,568,2020-01-01,321082.0,2196554.0,2196554.0,128369.03631,321079.0,2134729.000,126863.733692,-0.000,-0.028,-0.012
631,568,2020-02-01,314490.0,2131981.0,2131981.0,124592.96964,316838.0,2075608.625,123351.704144,0.007,-0.026,-0.010
632,568,2020-03-01,315461.0,2167222.0,2167222.0,126652.45368,317817.0,2084562.250,123881.899994,0.007,-0.038,-0.022
633,568,2020-04-01,307610.0,2104703.0,2104703.0,122998.84332,309777.0,2023535.875,120254.406723,0.007,-0.039,-0.022
634,568,2020-05-01,248270.0,1706966.0,1706966.0,99755.09304,250072.0,1641259.375,97538.890682,0.007,-0.038,-0.022
635,568,2020-06-01,319323.0,2150060.0,2150060.0,125649.50640,321356.0,2106613.750,125191.999947,0.006,-0.020,-0.004
636,568,2020-07-01,334439.0,2289373.0,2289373.0,135681.10476,334439.0,2235356.000,134392.040539,0.000,-0.024,-0.010
637,568,2020-08-01,313658.0,2145056.0,2145056.0,125372.67174,315360.0,2080005.375,123619.218920,0.005,-0.030,-0.014
638,568,2020-09-01,297688.0,2033831.0,2033831.0,118877.91504,298855.0,1977724.875,117547.462650,0.004,-0.028,-0.011
639,568,2020-10-01,192534.0,1323805.0,1323805.0,77370.25470,193806.0,1288821.125,76592.778698,0.007,-0.026,-0.010


# Adjust emissions
We next need to make certain adjustments to the data:
 - [ ] Calculate emissions for Geothermal plants
 - [ ] Adjust heat input/emissions from CHP plants by proportion used for electric generation
 - EPA adjusts biomass emissions, but not sure if we want to do that. Need to look into it more

In [142]:
# try using the method we used to impute net generation - adjust emissions based on proportion of heat used for electricity vs steam

# for now, let's just add a placeholder column
cems['co2_mass_tons_adjusted'] = cems['co2_mass_tons']

# Compare results to eGRID totals

Before we allocate the data to the hourly level, we should double check that the total annual emissions / generation values match the "official" data published in eGRID at the annual level.

In [143]:
# Aggregate total calculated values
###################################

# combine cems and eia data
cems_plant_annual = cems.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum()[['net_generation_mwh','heat_content_mmbtu','co2_mass_tons']].rename(columns={'heat_content_mmbtu':'heat_input_mmbtu'}).reset_index()
eia_plant_annual = monthly_eia_data_to_distribute.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum()[['net_generation_mwh','fuel_consumed_mmbtu','co2_mass_tons']].rename(columns={'fuel_consumed_mmbtu':'heat_input_mmbtu'}).reset_index()

plant_annual_total = pd.concat([cems_plant_annual,eia_plant_annual], axis=0)

# group any plants that have records from both datasets
plant_annual_total = plant_annual_total.groupby(['ba_code','state','plant_id_eia'], dropna=False).sum().reset_index()

# For plants that have different EPA and EIA plant IDs, the plant ID in eGRID is usually the EPA ID, but sometimes the EIA ID
# however, there are sometime 2 EIA IDs for a single eGRID ID, so we need to group the data in the EIA table by the egrid id
# We need to update all of the egrid plant IDs to the EIA plant IDs
egrid_crosswalk = pd.read_csv('../data/egrid/egrid_static_tables/2020/table_C5_crosswalk_of_EIA_ID_to_EPA_ID.csv')
eia_to_egrid_id = dict(zip(list(egrid_crosswalk['plant_id_eia']), list(egrid_crosswalk['plant_id_egrid'])))
egrid_to_eia_id = dict(zip(list(egrid_crosswalk['plant_id_egrid']), list(egrid_crosswalk['plant_id_eia'])))
plant_annual_total['plant_id_egrid'] = plant_annual_total['plant_id_eia']
plant_annual_total['plant_id_egrid'].update(plant_annual_total['plant_id_egrid'].map(eia_to_egrid_id))

# Load the eGRID plant table
############################

# load plant level data from egrid
egrid_plant = pd.read_excel(f'../data/egrid/egrid{year}_data.xlsx', 
                            sheet_name=f'PLNT{str(year)[-2:]}', 
                            header=1, 
                            usecols=['BACODE','PSTATABB', 'PLPRMFL','ORISPL', 'PNAME','PLGENATN', 'PLGENATR', 'PLHTIANT','UNCO2','PLCO2AN'])
# calculate total net generation from reported renewable and nonrenewable generation
egrid_plant['net_generation_mwh'] = egrid_plant['PLGENATN'] + egrid_plant['PLGENATR']
egrid_plant = egrid_plant.drop(columns=['PLGENATN', 'PLGENATR'])
# rename the columns
egrid_plant = egrid_plant.rename(columns={'BACODE':'ba_code',
                                          'PSTATABB':'state',
                                          'PLPRMFL':'energy_source_code',
                                          'ORISPL':'plant_id_egrid',
                                          'PNAME':'plant_name',
                                          'PLHTIANT':'heat_input_mmbtu',
                                          'UNCO2':'co2_mass_tons',
                                          'PLCO2AN':'co2_mass_tons_adjusted'})

# if egrid has a missing value for co2 for a clean plant, replace with zero
clean_fuels = ['SUN','MWH','WND', 'WAT','WH','PUR','NUC']
egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons_adjusted'] = egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons_adjusted'].fillna(0)
egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons'] = egrid_plant.loc[egrid_plant['energy_source_code'].isin(clean_fuels), 'co2_mass_tons'].fillna(0)

# reorder the columns                                         
egrid_plant = egrid_plant[['ba_code', 'state', 'plant_id_egrid', 'plant_name', 'net_generation_mwh', 'heat_input_mmbtu', 'co2_mass_tons', 'co2_mass_tons_adjusted']]

# remove any plants that habe no reported data
# NOTE: it seems that egrid includes a lot of proposed projects that are not yet operating, but just has missing data for them
plants_with_no_data_in_egrid = list(egrid_plant[egrid_plant[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','co2_mass_tons_adjusted']].sum(axis=1) == 0]['plant_id_egrid'])
egrid_plant = egrid_plant[~egrid_plant['plant_id_egrid'].isin(plants_with_no_data_in_egrid)]

# We also want to remove any plants that are located in Puerto Rico
egrid_plant = egrid_plant[(egrid_plant['state'] != 'PR')]

# create a column for eia id
egrid_plant['plant_id_eia'] = egrid_plant['plant_id_egrid']
egrid_plant['plant_id_eia'].update(egrid_plant['plant_id_eia'].map(egrid_to_eia_id))

## Identify plants that are missing from each dataset
Notes as of 4/21/22:
 - plant 7922 is being removed in gen_fuel_allocated because it is marked as a retired plant, and only has generation in the gf table
 - most of these other plants are being removed because they report 0 net generation, even though they have fuel consumption for electricity
 - not sure if these plants are exporting electricity to the grid. If not, it is probably safe to leave them off of the grid average emissions calculation

In [144]:
# identify any plants that are in egrid but not our totals, and any plants that are in our totals, but not egrid
plant_not_in_calc = list(set(egrid_plant['plant_id_eia'].unique()) - set(plant_annual_total['plant_id_eia'].unique()))
plants_not_in_egrid = list(set(plant_annual_total['plant_id_egrid'].unique()) - set(egrid_plant['plant_id_egrid'].unique()))

# Which plants are included in eGRID but are missing from our calculations?
missing_from_calc = egrid_plant[egrid_plant['plant_id_egrid'].isin(plant_not_in_calc)]

#missing_from_calc.to_csv('../data/temp/plants_missing_from_calcs.csv', index=False)

# see if any of these plants are retired
generators_eia860 = load_data.load_pudl_table('generators_eia860', year=year)
missing_from_calc.merge(generators_eia860.groupby('plant_id_eia')['retirement_date'].unique().reset_index(), how='left', on='plant_id_eia')

,ba_code,state,plant_id_egrid,plant_name,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_adjusted,plant_id_eia,retirement_date
0,CISO,CA,328,Borel,-204.0,NaN,0.0,0.0,328,[2017-09-01]
1,MISO,IA,7922,Brooklyn City North Plant,42.0,NaN,NaN,NaN,7922,[2011-09-01]


In [145]:
# Which plants are in our calculations, but are missing from eGRID?
plant_names = load_data.load_pudl_table('plants_entity_eia')[['plant_id_eia','plant_name_eia','sector_name_eia']]
missing_from_egrid = plant_annual_total[plant_annual_total['plant_id_eia'].isin(plants_not_in_egrid)].merge(plant_names, how='left', on='plant_id_eia')

missing_from_egrid

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,plant_id_egrid,plant_name_eia,sector_name_eia
0,AECI,MO,2127,0.0000,0.00,0.000000,2127,Gallatin (MO),Electric Utility
1,AECI,MO,2141,0.0000,0.00,0.000000,2141,Macon,Electric Utility
2,AECI,MO,2146,0.0000,0.00,0.000000,2146,Monroe (MO),Electric Utility
3,AECI,MO,8110,0.0000,0.00,0.000000,8110,Stanberry,Electric Utility
4,AECI,MO,56126,0.0000,0.00,0.000000,56126,Sub 2 Generating Station,Electric Utility
...,...,...,...,...,...,...,...,...,...
358,NaN,AK,60250,0.0000,441.00,36.012060,60250,Swampy Acres Microgrid,Electric Utility
359,NaN,HI,774,0.0000,0.00,0.000000,774,Waiau Hydro,Electric Utility
360,NaN,HI,63280,0.0000,0.00,0.000000,63280,"AES Kekaha Solar, LLC Hybrid",IPP Non-CHP
361,NaN,NaN,52106,0.0000,2379781.25,139074.420255,52106,Philadelphia Refinery,Industrial CHP


In [146]:
# how many of the plants missing from egrid have non-zero data
missing_from_egrid[missing_from_egrid[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons']].sum(axis=1) > 0]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,plant_id_egrid,plant_name_eia,sector_name_eia
68,CISO,CA,57714,0.000000e+00,1.00,5.844000e-02,57714,City of Palo Alto,Electric Utility
79,CISO,CA,61464,0.000000e+00,1276.00,1.041982e+02,61464,"CoreSite Real Estate 2972 Stender, L.P.",IPP Non-CHP
80,CISO,CA,61474,0.000000e+00,223.00,1.821018e+01,61474,CoreSite Real Estate 55 S. Market Street,Commercial Non-CHP
81,CISO,CA,61475,0.000000e+00,1270.00,1.037082e+02,61475,"CoreSite Real Estate 1656 McCarthy, L.P.",Commercial Non-CHP
82,CISO,CA,61476,0.000000e+00,584.00,4.768944e+01,61476,"CoreSite Real Estate 2901 Coronado, L.P.",Commercial Non-CHP
83,CISO,CA,61482,0.000000e+00,2249.00,1.836533e+02,61482,"CoreSite Real Estate 3032 Coronado, L.P.",Commercial Non-CHP
91,CPLE,NC,54316,0.000000e+00,16737.00,9.818023e+02,54316,Southport,Industrial CHP
97,DUK,NC,54363,0.000000e+00,204.00,1.665864e+01,54363,PPG Industries Inc Shelby NC W,Industrial Non-CHP
106,FMPP,FL,7997,0.000000e+00,3589.00,2.930777e+02,7997,Winston,Electric Utility
145,MISO,IA,1146,0.000000e+00,6.00,4.899600e-01,1146,Harlan,Electric Utility


In [147]:
# identify where there is a single egrid plant id for multiple eia plant ids
double_ids = plant_annual_total[plant_annual_total['plant_id_egrid'].duplicated(keep=False)]
double_ids = double_ids.groupby('plant_id_egrid').sum()['net_generation_mwh'].reset_index() # focus on net generation for now
# merge the egrid data
double_ids = double_ids.merge(egrid_plant[['plant_id_egrid','net_generation_mwh']], how='left', on='plant_id_egrid', suffixes=('_calc','_egrid'))
double_ids['percent_diff'] = ((double_ids['net_generation_mwh_calc'] - double_ids['net_generation_mwh_egrid']) / double_ids['net_generation_mwh_egrid']).round(3)
double_ids

,plant_id_egrid,net_generation_mwh_calc,net_generation_mwh_egrid,percent_diff
0,562,2.955757e+05,295534.003,0.0
1,762,1.313200e+04,13132.000,0.0
2,1416,3.497026e+06,3497026.000,0.0
3,2709,5.829410e+06,5829409.996,0.0
4,3612,4.045388e+06,4045388.003,-0.0
5,4076,1.105390e+05,110539.000,0.0
6,10474,6.704110e+05,670410.997,0.0
7,55306,8.487498e+06,8487498.000,0.0
8,55375,5.447419e+06,5447419.084,-0.0
9,55481,7.276336e+06,7276336.000,0.0


## Identify plants for which we are missing a BA assignment
(of the plants not already missing from our calculated totals)

In [148]:
ba_code_match = egrid_plant.set_index('plant_id_eia')[['plant_name','ba_code']].merge(plant_annual_total.set_index('plant_id_eia')[['ba_code']], how='inner', left_index=True, right_index=True, suffixes=("_egrid",'_calc'))

# plants with missing ba code
ba_code_match[(ba_code_match['ba_code_calc'].isna()) & ~(ba_code_match['ba_code_egrid'].isna())]

,plant_name,ba_code_egrid,ba_code_calc
plant_id_eia,,,


## Identify plants for which we have incorrectly assigned the BA code

In [149]:
# plants with incorrect ba code
ba_code_match[(ba_code_match['ba_code_calc'] != ba_code_match['ba_code_egrid']) & ~(ba_code_match['ba_code_calc'].isna())]

,plant_name,ba_code_egrid,ba_code_calc
plant_id_eia,,,


## Identify where our calculated totals do not match eGRID's totals

In [156]:
# standardize column names and index so that the two dfs can be divided
calculated_to_compare = plant_annual_total.groupby('plant_id_egrid').sum().drop(columns=['plant_id_eia'])

# create an adjusted co2 column
# TODO: remove this once we calculate adjusted emissions
calculated_to_compare['co2_mass_tons_adjusted'] = calculated_to_compare['co2_mass_tons']

# drop the plants that have no data in eGRID
egrid_plant = egrid_plant[~egrid_plant['plant_id_eia'].isin(plants_with_no_data_in_egrid)]

egrid_to_compare = egrid_plant.set_index(['plant_id_egrid']).drop(columns=['ba_code','state','plant_name','plant_id_eia'])

# divide calculated value by egrid value
compared = calculated_to_compare.div(egrid_to_compare).merge(egrid_plant[['plant_id_egrid','plant_name','ba_code', 'state']], how='left', left_index=True, right_on='plant_id_egrid').set_index('plant_id_egrid')
compared['plant_name'] = compared['plant_name'].fillna('unknown')

# create a dataframe that merges the two sources of data together
compared_merged = calculated_to_compare.merge(egrid_to_compare, how='outer', on='plant_id_egrid', suffixes=('_calc','_egrid'))

# for each column, change missing values to zero if both values are zero (only nan b/c divide by zero)
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons_adjusted','co2_mass_tons']:
    # identify plants with zero values for both
    plant_ids = list(compared_merged[(compared_merged[f'{col}_calc'] == 0) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plant_ids), col] = 1

# for each column, categorize the data based on how far it is off from egrid
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons_adjusted','co2_mass_tons']:
    # add a new column
    compared[f'{col}_status'] = pd.cut(x=compared[col], 
                                       bins=[-999999999,0,0.5,0.9,0.99,0.9999,1,1.0001,1.01,1.1,1.5,999999999], 
                                       labels=['negative','<50%','+/-50%','+/-10%','+/-1%','!exact','!exact','+/-1%','+/-10%','+/-50%','>50%'], 
                                       ordered=False)
    # replace any missing values with missing
    compared[f'{col}_status'] = compared[f'{col}_status'].astype(str)  
    compared[f'{col}_status'] = compared[f'{col}_status'].fillna('missing')
    compared[f'{col}_status'] = compared[f'{col}_status'].replace('nan','missing')
    compared.loc[(compared.index.isin(plants_not_in_egrid)),f'{col}_status'] = 'not_in_egrid'

# identify which plants are missing from egrid vs calculated values
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons_adjusted','co2_mass_tons']:
    # identify plants that are missing in egrid
    plants_missing_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] > 0) & (compared_merged[f'{col}_egrid'].isna())].index)
    compared.loc[compared.index.isin(plants_missing_egrid), f'{col}_status'] = 'missing_in_egrid'
    # identify plants that are missing from our calculations
    plants_missing_calc = list(compared_merged[(compared_merged[f'{col}_calc'].isna()) & (compared_merged[f'{col}_egrid'] > 0)].index)
    compared.loc[compared.index.isin(plants_missing_calc), f'{col}_status'] = 'missing_in_calc'
    # identify where our calculations are missing a zero value
    plants_missing_zero_calc = list(compared_merged[(compared_merged[f'{col}_calc'].isna()) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plants_missing_zero_calc), f'{col}_status'] = 'missing_zero_in_calc'
    # identify where egrid has a missing value instead of a zero
    plants_missing_zero_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] == 0) & (compared_merged[f'{col}_egrid'].isna())].index)
    compared.loc[compared.index.isin(plants_missing_zero_egrid), f'{col}_status'] = 'missing_zero_in_egrid'
    # identify where egrid has a zero value where we have a positive value
    plants_incorrect_zero_egrid = list(compared_merged[(compared_merged[f'{col}_calc'] > 0) & (compared_merged[f'{col}_egrid'] == 0)].index)
    compared.loc[compared.index.isin(plants_incorrect_zero_egrid), f'{col}_status'] = '>50%'

# create a dataframe that counts how many plants are in each category
comparison_count = []
for col in ['net_generation_mwh','heat_input_mmbtu','co2_mass_tons_adjusted','co2_mass_tons']:
    count = compared.groupby(f'{col}_status', dropna=False).count()['plant_name'].rename(col)
    count.index = count.index.rename('status')
    comparison_count.append(count)

comparison_count = pd.concat(comparison_count, axis=1).fillna(0).astype(int)
comparison_count = pd.concat([comparison_count, pd.DataFrame(comparison_count.sum().rename('Total')).T], axis=0)   
comparison_count

,net_generation_mwh,heat_input_mmbtu,co2_mass_tons_adjusted,co2_mass_tons
!exact,9863,8115,7413,8283
+/-1%,22,596,641,797
+/-10%,48,305,393,437
+/-50%,54,269,316,282
<50%,11,5,3,9
>50%,7,741,1152,110
missing,1,2,1,1
missing_in_calc,2,1,1,1
missing_in_egrid,3,58,165,165
missing_zero_in_egrid,361,324,328,328


In [157]:
print(comparison_count.to_markdown())

|                       |   net_generation_mwh |   heat_input_mmbtu |   co2_mass_tons_adjusted |   co2_mass_tons |
|:----------------------|---------------------:|-------------------:|-------------------------:|----------------:|
| !exact                |                 9863 |               8115 |                     7413 |            8283 |
| +/-1%                 |                   22 |                596 |                      641 |             797 |
| +/-10%                |                   48 |                305 |                      393 |             437 |
| +/-50%                |                   54 |                269 |                      316 |             282 |
| <50%                  |                   11 |                  5 |                        3 |               9 |
| >50%                  |                    7 |                741 |                     1152 |             110 |
| missing               |                    1 |                  2 |           

In [158]:
# examine specific plants in a category
value = 'net_generation_mwh'
status = '>50%'

#compared_merged.loc[64877,:]

compared[compared[f'{value}_status'] == status].sort_values(by=value)

,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_adjusted,plant_name,ba_code,state,net_generation_mwh_status,heat_input_mmbtu_status,co2_mass_tons_adjusted_status,co2_mass_tons_status
plant_id_egrid,,,,,,,,,,,
1599,1.626186,0.996860,0.996665,0.996665,Canal Station,ISNE,MA,>50%,+/-1%,+/-1%,+/-1%
2707,1.642643,1.003219,1.095939,1.095939,Blewett,CPLE,NC,>50%,+/-1%,+/-10%,+/-10%
335,1.856021,0.987818,0.987818,0.987818,AES Huntington Beach,CISO,CA,>50%,+/-10%,+/-10%,+/-10%
7288,2.060279,2.016973,1.999260,1.999260,Sherman Avenue,PJM,NJ,>50%,>50%,>50%,>50%
60698,2.154015,0.993232,0.992273,0.992273,Stanton Energy Reliability Center,CISO,CA,>50%,+/-1%,+/-1%,+/-1%
58256,2.660714,1.000000,1.000000,1.000000,Los Alamos PV Site,PNM,NM,>50%,!exact,!exact,!exact
52089,5.031182,8.388590,0.999923,8.387940,Celanese Acetate LLC,PJM,VA,>50%,>50%,>50%,!exact


## Notes

Net generation < 50%:
 - 2617 has negative net generation
 - 10444, 59395: some plants are hybrid fossil / solar plants, but the primary energy source code is getting listed as PV for certain generators, which is causing an issue

Net generation > 50%:
 - for plants where net generation ratio = 2.00, the generation was imputed, so the net generation should match exactly. Ignore these. (10867, 50900)
 - plant 2504 has three units (120, 121, 122) that don't appear in EIA, and in CEMS only report steam. 
 - 335 has two extra units (CT1 and CT2) that report to CEMS but not EIA. Likely correct
 - 1404 reports generation to CEMS in December, but not to EIA. This is likely correct
 - 7288 reports DFO to CEMS, but NG to EIA - there's probably some generators not reporting
 - 52193 should be fixed once steam allocation fixed
 - 60698 double counting generation from cEMS because of 90% heat threhshold in a month - might be fixed once we tackle heat input
 - 58256 is a solar/hybrid and in generators_entity_eia the battery portion is associated with a separate generator id (should only be 1)
 - 50933 the net generation reported is not being allocated correctly in EIA

Other:
 - plant 3754 has heat input in cems and eia that don't match
 - 2401 has generation in both cems and eia
 - 50933 might have allocation issue (doesn't appear in CEMS)


In [159]:
egrid_plant[egrid_plant['plant_id_eia'] == 50933]

,ba_code,state,plant_id_egrid,plant_name,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,co2_mass_tons_adjusted,plant_id_eia
12476,MISO,WI,50933,Rhinelander Mill,3045.0,129378.81,169927.852,1387.709,50933


In [160]:
plant_annual_total[plant_annual_total['plant_id_eia'] == 50933]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,plant_id_egrid
5984,MISO,WI,50933,3045.0,2283310.0,169926.4344,50933


In [126]:
gen_fuel_allocated[gen_fuel_allocated['plant_id_eia'] == 50933]

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_tons,ba_code,state,energy_source_code,data_source
126440,50933,GEN5,2020-01-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126441,50933,GEN5,2020-02-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126442,50933,GEN5,2020-03-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126443,50933,GEN5,2020-04-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126444,50933,GEN5,2020-05-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126445,50933,GEN5,2020-06-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126446,50933,GEN5,2020-07-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126447,50933,GEN5,2020-08-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126448,50933,GEN5,2020-09-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only
126449,50933,GEN5,2020-10-01,NaN,NaN,NaN,NaN,MISO,WI,NG,eia_only


In [72]:
cems_unit_monthly = cems.groupby(['plant_id_eia','unitid','report_date']).sum().reset_index()

In [122]:
cems_unit_monthly[cems_unit_monthly['plant_id_eia'] == 50933]

,plant_id_eia,unitid,report_date,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,plant_id_epa,net_generation_mwh,gross_to_net_ratio,co2_mass_tons_adjusted


In [123]:
cems[cems['plant_id_eia'] == 50933]

,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,...,plant_id_epa,unit_id_epa,report_date,energy_source_code,net_generation_mwh,gross_to_net_ratio,net_gen_method,ba_code,state,co2_mass_tons_adjusted


In [74]:
cems_plant_annual[cems_plant_annual['plant_id_eia'] == 10867]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons
453,MISO,IL,10867,386348.0,4476574.0,478769.577792


In [27]:
cems[cems['plant_id_eia'] == 61242]['cems_id'].unique()

array(['61242_CT1', '61242_CT2'], dtype=object)

In [64]:
cems_plant_annual[cems_plant_annual['net_generation_mwh'] == 0]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons
108,CISO,CA,57073,0.0,4.767792e+05,2.814200e+04
109,CISO,CA,57074,0.0,4.847860e+05,2.860230e+04
110,CISO,CA,57075,0.0,4.808632e+05,2.837380e+04
125,CPLE,NC,2707,0.0,7.067434e+03,5.720210e+02
127,CPLE,NC,2716,0.0,1.681283e+04,1.361799e+03
...,...,...,...,...,...,...
1245,SWPP,OK,10671,0.0,1.097756e+07,1.146946e+06
1297,TVA,MS,55076,0.0,2.602014e+07,2.829756e+06
1311,TVA,TN,50296,0.0,6.330312e+05,7.015885e+04
1312,TVA,TN,50956,0.0,1.904276e+06,1.112859e+05


In [52]:
monthly_eia_data_to_distribute[monthly_eia_data_to_distribute['plant_id_eia'] == 2878]

,plant_id_eia,generator_id,report_date,net_generation_mwh,fuel_consumed_mmbtu,co2_mass_tons,ba_code,state,energy_source_code_1,data_source
54665,2878,CT,2020-01-01,6.523,175.0,14.29050,PJM,OH,DFO,eia_only
54666,2878,CT,2020-02-01,8.268,204.0,16.65864,PJM,OH,DFO,eia_only
54667,2878,CT,2020-03-01,5.057,123.0,10.04418,PJM,OH,DFO,eia_only
54668,2878,CT,2020-04-01,5.608,140.0,11.43240,PJM,OH,DFO,eia_only
54669,2878,CT,2020-05-01,7.415,181.0,14.78046,PJM,OH,DFO,eia_only
54670,2878,CT,2020-06-01,8.124,198.0,16.16868,PJM,OH,DFO,eia_only
54671,2878,CT,2020-07-01,11.762,292.0,23.84472,PJM,OH,DFO,eia_only
54672,2878,CT,2020-08-01,7.017,175.0,14.29050,PJM,OH,DFO,eia_only
54673,2878,CT,2020-09-01,4.330,105.0,8.57430,PJM,OH,DFO,eia_only
54674,2878,CT,2020-10-01,9.504,233.0,19.02678,PJM,OH,DFO,eia_only


In [174]:
# there are some plants that report heat input and co2 in CEMS, but are missing net generation data
# TODO: we should maybe try and fill net generation data using EIA-923?
cems_plant_annual[cems_plant_annual['net_generation_mwh'] == 0]

,ba_code,state,plant_id_eia,net_generation_mwh,heat_input_mmbtu,co2_mass_tons
108,CISO,CA,57073,0.0,4.767792e+05,28141.999997
109,CISO,CA,57074,0.0,4.847860e+05,28602.299972
110,CISO,CA,57075,0.0,4.808632e+05,28373.799968
125,CPLE,NC,2707,0.0,7.067434e+03,572.020997
127,CPLE,NC,2716,0.0,1.681283e+04,1361.799019
265,FPC,FL,7294,0.0,5.400395e+03,321.151797
319,ISNE,CT,56189,0.0,1.051138e+04,858.359701
374,ISNE,VT,3734,0.0,3.513840e+03,284.729996
375,ISNE,VT,3754,0.0,7.887080e+03,639.309984
747,PJM,DE,591,0.0,1.516640e+03,122.799997


In [172]:
cems[cems['plant_id_eia'] == 57073]

,plant_id_eia,unitid,cems_id,operating_datetime_utc,operating_time_hours,gross_load_mw,gross_generation_mwh,steam_load_1000_lbs,heat_content_mmbtu,co2_mass_tons,...,plant_id_epa,unit_id_epa,report_date,cems_reporting_category,energy_source_code,ba_code,state,net_generation_mwh,gross_to_net_ratio,net_gen_method
1749864,57073,BLR1,57073_BLR1,2020-01-01 08:00:00+00:00,1.0,0.0,0.0,NaN,51.900002,3.1,...,57073,90614,2020-01-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
1749865,57073,BLR1,57073_BLR1,2020-01-01 09:00:00+00:00,1.0,0.0,0.0,NaN,53.000000,3.1,...,57073,90614,2020-01-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
1749866,57073,BLR1,57073_BLR1,2020-01-01 10:00:00+00:00,1.0,0.0,0.0,NaN,51.900002,3.1,...,57073,90614,2020-01-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
1749867,57073,BLR1,57073_BLR1,2020-01-01 11:00:00+00:00,1.0,0.0,0.0,NaN,51.900002,3.1,...,57073,90614,2020-01-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
1749868,57073,BLR1,57073_BLR1,2020-01-01 12:00:00+00:00,1.0,0.0,0.0,NaN,51.900002,3.1,...,57073,90614,2020-01-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542323,57073,BLR1,57073_BLR1,2021-01-01 03:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.0,...,57073,90614,2020-12-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
3542324,57073,BLR1,57073_BLR1,2021-01-01 04:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.0,...,57073,90614,2020-12-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
3542325,57073,BLR1,57073_BLR1,2021-01-01 05:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.0,...,57073,90614,2020-12-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio
3542326,57073,BLR1,57073_BLR1,2021-01-01 06:00:00+00:00,0.0,0.0,0.0,NaN,0.000000,0.0,...,57073,90614,2020-12-01,full_year,SUN,CISO,CA,0.0,inf,monthly_ratio


## Compare data at BA level

In [161]:
# load egrid BA totals
egrid_ba = pd.read_excel(f'../data/egrid/egrid{year}_data.xlsx', sheet_name=f'BA{str(year)[-2:]}', header=1, usecols=['BANAME','BACODE','BAHTIANT','BANGENAN','BACO2AN'])
# rename the columns
egrid_ba = egrid_ba.rename(columns={'BANAME':'ba_name',
                                    'BACODE':'ba_code',
                                    'BAHTIANT':'heat_input_mmbtu',
                                    'BANGENAN':'net_generation_mwh',
                                    'BACO2AN':'co2_mass_tons'})

# divide our calculation by the BA totals from eGRID
# if there are 0 values, replace with 0.1, so that div by zero doesn't return missing value
ba_metric = plant_annual_total.groupby('ba_code', dropna=False).sum().drop(columns='plant_id_eia').replace(0,0.1).div(egrid_ba.set_index('ba_code').drop(columns='ba_name').replace(0,0.1)).sort_values(by='co2_mass_tons').round(3)

total = pd.DataFrame(plant_annual_total[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons']].sum().div(egrid_ba[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons']].sum()).rename('Total')).T

# calculate the difference in the number of plants in each region
plant_count = (plant_annual_total.groupby('ba_code', dropna=False).count()['plant_id_egrid'] - egrid_plant.groupby('ba_code', dropna=False).count()['plant_id_egrid']).rename('num_plants')
ba_metric = ba_metric.merge(plant_count, how='left', left_index=True, right_index=True).drop(columns=['plant_id_egrid'])

ba_metric = pd.concat([ba_metric, total], axis=0) 

ba_metric = ba_metric[['net_generation_mwh','heat_input_mmbtu','co2_mass_tons','num_plants']]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ba_metric)

,net_generation_mwh,heat_input_mmbtu,co2_mass_tons,num_plants
NaN,0.309000,0.349000,0.298000,9.0
TEPC,0.697000,0.818000,0.859000,0.0
PGE,1.000000,0.916000,0.913000,5.0
DEAA,1.000000,0.997000,0.997000,0.0
TIDC,1.000000,0.998000,0.998000,0.0
GRIF,1.000000,0.999000,0.999000,0.0
CSTO,1.000000,1.000000,1.000000,0.0
OVEC,1.000000,1.000000,1.000000,0.0
LGEE,1.000000,1.000000,1.000000,0.0
NSB,1.000000,1.000000,1.000000,0.0


In [162]:
print(ba_metric.to_markdown())

|       |   net_generation_mwh |   heat_input_mmbtu |   co2_mass_tons |   num_plants |
|:------|---------------------:|-------------------:|----------------:|-------------:|
| nan   |             0.309    |            0.349   |          0.298  |            9 |
| TEPC  |             0.697    |            0.818   |          0.859  |            0 |
| PGE   |             1        |            0.916   |          0.913  |            5 |
| DEAA  |             1        |            0.997   |          0.997  |            0 |
| TIDC  |             1        |            0.998   |          0.998  |            0 |
| GRIF  |             1        |            0.999   |          0.999  |            0 |
| CSTO  |             1        |            1       |          1      |            0 |
| OVEC  |             1        |            1       |          1      |            0 |
| LGEE  |             1        |            1       |          1      |            0 |
| NSB   |             1        |           

# Assign monthly data to hourly profile
We now, in theory, have complete data on national-level heat input, net generation, and emissions, from a combination of two sources:
    1. hourly data from CEMS
    2. momthly data for generators that don't report to CEMS

For the second category of monthly data, we need to figure out how to allocate the monthly level data to each hour. 

In [ ]:
# categorize generators by broad fuel categories (clean, geothermal, biofuel, fossil)
clean_fuels = ['SUN','MWH','WND', 'WAT','WH','PUR','NUC']
fossil_fuels = ['NG', 'DFO','OG','WDS','BIT','OTH','PC','SUB', 'LIG','KER', 'RC', 'WO','RFO', 'WC', 'SGC', 'SGP', 'PG', 'JF','BFG']
bio_fuels = ['AB','BG','BLQ','DG','LFG','MSB','MSW','OBG','OBL','OBS','SLW','WDL','WDS']
geo_fuel = ['GEO']
# TODO: Figure out what to do with MSW

# Output data 

Save data to a CSV so we can separate generation of and analysis of hourly e-grid numbers